In [ ]:
%load_ext autoreload
%autoreload 1
import sys; sys.path.append("..")
%aimport lib.util
from lib.util import reformat_tract, add_race_eth, add_purpose_type, add_income_group, add_all_category_columns
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1500
import numpy as np
from ambry import library
l = library()
p = l.get('ffiec.gov-hmda-2010e-lar-2012-ca').partition
df = p.select("SELECT * FROM lar WHERE county_code = 73").pandas

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
add_race_eth(df) # Adds race_eth column
add_purpose_type(df) # Adds purpose_type column Combines loan_purpose and loan_type in the same way as the summary report
add_income_group(df) # Adds income_group column
add_all_category_columns(df) # Add '_c' categorical columns



# All Lenders Combined

In [ ]:
###
# Action type == ALL
###
dfa=df
totalcount = dfa.loan_amount.count()
totalamount = dfa.loan_amount.sum()

### By Loan Purpose and Type
loanpt = dfa.groupby(['purpose_type'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity
race_eth  = dfa.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income
income  = dfa.groupby(['income_group'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex
sex  = dfa.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together
frames = [loanpt, race_eth,income,sex]
totalapp = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
totalapp
pieces = [totalapp.ix[:, ["count","%count","amount","%amount",]]]
totalapp = pd.concat(pieces, axis=1, keys=['Total Applications (1)'])

In [ ]:
#######
# Action Type =1
#######
dfa = df[df.action_type==1]
totalcount = dfa.loan_amount.count()
totalamount = dfa.loan_amount.sum()

### By Loan Purpose and Type
loanpt = dfa.groupby(['purpose_type'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity
race_eth  = dfa.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income
income  = dfa.groupby(['income_group'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex
sex  = dfa.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together
frames = [loanpt, race_eth,income,sex]
totalapp1 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp1.ix[:, ["count","%count","amount","%amount",]]]
totalapp1 = pd.concat(pieces, axis=1, keys=['Action Type = 1'])

#######
# Action Type =2
#######
dfa = df[df.action_type==2]
totalcount = dfa.loan_amount.count()
totalamount = dfa.loan_amount.sum()

### By Loan Purpose and Type
loanpt = dfa.groupby(['purpose_type'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity
race_eth  = dfa.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income
income  = dfa.groupby(['income_group'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex
sex  = dfa.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together
frames = [loanpt, race_eth,income,sex]
totalapp2 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp2.ix[:, ["count","%count","amount","%amount",]]]
totalapp2 = pd.concat(pieces, axis=1, keys=['Action Type = 2'])

#######
# Action Type =3
#######
dfa = df[df.action_type==3]
totalcount = dfa.loan_amount.count()
totalamount = dfa.loan_amount.sum()

### By Loan Purpose and Type
loanpt = dfa.groupby(['purpose_type'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity
race_eth  = dfa.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income
income  = dfa.groupby(['income_group'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex
sex  = dfa.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together
frames = [loanpt, race_eth,income,sex]
totalapp3 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp3.ix[:, ["count","%count","amount","%amount",]]]
totalapp3 = pd.concat(pieces, axis=1, keys=['Action Type = 3'])

#######
# Action Type =4
#######
dfa = df[df.action_type==4]
totalcount = dfa.loan_amount.count()
totalamount = dfa.loan_amount.sum()

### By Loan Purpose and Type
loanpt = dfa.groupby(['purpose_type'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity
race_eth  = dfa.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income
income  = dfa.groupby(['income_group'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex
sex  = dfa.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together
frames = [loanpt, race_eth,income,sex]
totalapp4 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp4.ix[:, ["count","%count","amount","%amount",]]]
totalapp4 = pd.concat(pieces, axis=1, keys=['Action Type = 4'])

#######
# Action Type =5
#######
dfa = df[df.action_type==5]
totalcount = dfa.loan_amount.count()
totalamount = dfa.loan_amount.sum()

### By Loan Purpose and Type
loanpt = dfa.groupby(['purpose_type'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity
race_eth  = dfa.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income
income  = dfa.groupby(['income_group'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex
sex  = dfa.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together
frames = [loanpt, race_eth,income,sex]
totalapp5 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp5.ix[:, ["count","%count","amount","%amount",]]]
totalapp5 = pd.concat(pieces, axis=1, keys=['Action Type = 5'])

#######
# Action Type = 6
#######
dfa = df[df.action_type==6]
totalcount = dfa.loan_amount.count()
totalamount = dfa.loan_amount.sum()

### By Loan Purpose and Type
loanpt = dfa.groupby(['purpose_type'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity
race_eth  = dfa.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income
income  = dfa.groupby(['income_group'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex
sex  = dfa.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together
frames = [loanpt, race_eth,income,sex]
totalapp6 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp6.ix[:, ["count","%count","amount","%amount",]]]
totalapp6 = pd.concat(pieces, axis=1, keys=['Action Type = 6'])

In [ ]:
totalapp1

In [ ]:
all_lenders = pd.concat([totalapp,totalapp1,totalapp2,totalapp3,totalapp4,totalapp5,totalapp6], axis=1)
all_lenders

In [ ]:
all_lenders.to_csv('allLenders.csv')